# Oogeso - test case 2
Simple case for illustrating/testing the Oogeso model

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import IPython
import pyomo.environ as pyo
import logging
import pandas as pd
import plotly.express as px
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import oogeso
#from oogeso import plots
oogeso.plots.plotter='plotly'

In [ ]:
data0 = oogeso.file_io.read_data_from_yaml('test case2.yaml')
profiles = oogeso.file_io.read_profiles_from_xlsx('test case2 profiles.xlsx')
store_duals = {
    'elcost':{'constr':'constrDevicePmin','indx':('dem',None)},
    #'wind': {'constr':'constrTerminalEnergyBalance','indx':('el','windfarm','out',None)},
    }

# MODIFY input data
for dev in ['GT1','GT2']:
    data0['paramDevice'][dev]['isOn_init'] = 1
#data['paramParameters']['elReserveMargin'] = 0

data = oogeso.file_io.create_initdata(data0)
mc = oogeso.Multicarrier(loglevel="INFO")
mc.createModelInstance(data,profiles)

In [ ]:
IPython.display.Image(oogeso.plots.plotNetwork(mc,timestep=None).create_png())

In [ ]:
status = mc.solveMany(solver="cbc",timerange=[0,90],write_yaml=False,
                     store_duals=store_duals)

In [ ]:
#mc.instance.constrDevice_startup_delay._data['GT2',0].pprint()
#mc.instance.constrDevice_startup_delay.clear()
#mc.instance.constrDevice_startup_delay.construct()
#mc.instance.constrDevice_startup_delay._data
#mc.instance.constrDevice_startup_delay.reconstruct()
mc.instance.constrTerminalEnergyBalance['el','node1','out',0].pprint()
mc.instance.constrTerminalEnergyBalance['el','node1','in',0].pprint()
#mc.instance.constrDevice_gasturbine.pprint()
mc.instance.constrDevice_ramprate['GT2',1].pprint()

#mc.instance.pprint()

In [ ]:
gt='GT2'
print("GT is on initially ={}".format(pyo.value(mc.instance.paramDeviceIsOnInitially[gt])))
print("GT timesteps in startup preparation = {}".format(pyo.value(mc.instance.paramDevicePrepTimestepsInitially[gt])))
for t in range(4):
    cstr=mc.instance.constrDevice_startup_shutdown[gt,t]
    print("t={}: {} = {}".format(t,cstr.body,cstr.upper))
for t in range(4):
    cstr=mc.instance.constrDevice_startup_delay[gt,t]
    print("t={}: {} = {}".format(t,cstr.body,cstr.upper))
for t in range(4):
    cstr=mc.instance.constrDevicePmax[gt,t]
    print("t={}: {} = {}".format(t,cstr.body,cstr.upper))

In [ ]:
dotG=oogeso.plots.plotNetwork(mc,timestep=15,filename=None)
#IPython.display.SVG(dotG.create_svg()) #original size, vector graphics
IPython.display.Image(dotG.create_png()) #scales automatically to page width

In [ ]:
oogeso.plots.plot_SumPowerMix(mc,carrier="el").show()

In [ ]:
fig=px.line(mc._dfDuals,title="Specific cost, marginal cost, dual value")
cost_co2=mc.instance.paramParameters['co2_tax'] #NOK/kg
gts=['GT1','GT2','GT3']
# GTG specific cost (cost per power output)
gtg_P=mc._dfDeviceFlow[:,'el','out',:].unstack('device')[gts] # MJ/s=MW
gtg_co2=mc._dfCO2rate_per_dev[gts] # kg/s
gtg_cost = cost_co2*gtg_co2/gtg_P # NOK/MJ
gtg_avgcost = cost_co2*gtg_co2.sum(axis=1)/gtg_P.sum(axis=1) # NOK/kg x kg/s x 1/MW = NOK/MJ
for col in gtg_cost:
    fig.add_scatter(y=gtg_cost[col],x=gtg_cost.index, mode="lines",name=col+' specific cost')
fig.add_scatter(y=gtg_avgcost,x=gtg_avgcost.index, mode="lines",name='Average GTG specific cost',
                line={'dash':'dash'})

# GTG marginal costs:
for gt in gts:
    fuelrate_per_power = (mc.instance.paramDevice[gt]['fuelA']
                          /mc.instance.paramCarriers['gas']['energy_value']) #Sm3/MJ
    costrate = (mc.instance.paramParameters['co2_tax']
                *mc.instance.paramCarriers['gas']['CO2content']
                *fuelrate_per_power) # kr/MJ
    fig.add_scatter(y=[costrate,costrate],x=[gtg_avgcost.index[0],gtg_avgcost.index[-1]], 
                    mode="lines",name="{} marginal cost".format(gt))
fig.update_xaxes(title_text='Timestep')
fig.update_yaxes(title_text='Cost (NOK/MJ)')
fig.show()

In [ ]:
# Problem = isOn = isPrep, isStarting=0
gt='GT2'
dfStart=pd.concat([
    mc._dfDeviceStarting[gt],
    mc._dfDeviceIsPrep[gt],
    mc._dfDeviceIsOn[gt],
    mc._dfDeviceStopping[gt]],axis=1)
dfStart.columns=['starting','prep','on','stopping']
#dfStart[12:25]

In [ ]:
#mc.instance.varDeviceIsPrep.pprint()

In [ ]:
oogeso.plots.plot_deviceprofile(mc,devs=['GT1','GT2','GT3'],includeOnOff=True,includePrep=True)

In [ ]:
#plots.plot_CO2rate(mc)
oogeso.plots.plot_CO2rate_per_dev(mc)

In [ ]:
oogeso.plots.plotReserve(mc)

In [ ]:
oogeso.plots.plotElBackup(mc,showMargin=True)

In [ ]:
dfplot=mc._dfElReserve
margin=mc.instance.paramParameters['elReserveMargin']
x = [dfplot.index[0],dfplot.index[-1]]
fig=px.line(dfplot,title="Reserve")
fig.add_scatter(x=x,y=[margin,margin],line=dict(dash='dot'),name="margin",mode="lines")
fig.update_xaxes(title_text="Timestep")
fig.update_yaxes(title_text="Reserve power (MW)",dtick = 5)
fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0))

In [ ]:
if 'battery' in mc.instance.setDevice:
    fig=oogeso.plots.plot_devicePowerEnergy(mc,'battery',
                                 filename=None)
    fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0)).show()
    print("Last optimisation scheduling:")
    oogeso.milp_plot.plotDevicePowerLastOptimisation1(mc,device='battery',
                filename=None)